In [ ]:
!pip install torch

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os
import torch.optim as optim
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, data_dir, transform=None):
    self.data_dir = data_dir
    self.transform = transform

    self.samples = []
    self.labels = []

    for root, dirs, files in os.walk(data_dir):
      for filee in files:

        if filee.endswith(".jpg") or filee.endswith(".jpeg"):
          image_path = os.path.join(root, filee)

          label = filee.split("_")[1]
          self.labels.append(label)

  def __len__(self):
    return len(self.samples)

  def __getitem__(self, idx):

    image_path = self.samples[idx]
    image = Image.open(image_path)

    if self.transform:
      image = self.transform(image)

    label = self.labels[idx]

    return image, label

In [ ]:
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Resize((224, 224)), # Resize images to VGG input size
    transforms.ToTensor(),
])

custom_dataset = CustomDataset(data_dir='path_to_your_dataset', transform=transform)

train_size = int(0.8 * len(custom_dataset))
val_size = len(custom_dataset) - train_size

train_dataset, val_dataset = random_split(custom_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

In [ ]:
import torch
import torch.nn as nn

class VGG19(nn.Module):
    def __init__(self, num_classes):
        super(VGG19, self).__init__()
        self.features = nn.Sequential(
            # Convolutional layers
            nn.Conv2d(3, 64, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(512, 512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))

        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096), nn.ReLU(inplace=True), nn.Dropout(),
            nn.Linear(4096, 4096), nn.ReLU(inplace=True), nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
num_classes = 10
vgg19_model = VGG19(num_classes)
vgg19_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg19_model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
  vgg19_model.train()
  running_loss = 0.0

  for inputs, labels in train_loader:
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()

    outputs = vgg19_model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    train_loss = running_loss / len(train_loader)


    vgg19_model.eval()
    val_predictions = []
    val_targets = []

    with torch.no_grad():
      for inputs, labels in val_laoder:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = vgg19_model(inputs)
        _, predicted = torch.max(outputs, 1)

        val_predictions.extend(predicted.cpu().numpy())
        val_targets.extend(labels.cpu().numpy())

    val_f1 = f1_score(val_targets, val_predictions, average='weighted')
    val_conf_matrix = confusion_matrix(val_targets, val_predictions)

    print(f"Epoch [{epoch + 1 }/{num_epochs}] - "
          f"Train Loss: {train_loss:.4f}, "
          f"Val F1 Score: {val_f1:.4f}")
    print("Confusion Matrix:")
    print(val_conf_matrix)

print("Training Finished.")




plt.figure(figsize=(8, 6))
sns.heatmap(val_conf_matrix, annot=True, fmt='d', cmap="Blues",
            xticklabels=range(num_classes), yticklabels=range(num_classes))

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()